In [93]:
from lxml import etree
import cssselect 
import pandas as pd

In [94]:
parser = etree.XMLParser(remove_blank_text=True)

In [127]:
nomDir='/Users/gilles/Desktop/'
nomFichier="TEMP.html"
nom=nomDir+nomFichier
nomGroupes=nomDir+"TEMP-Groups.txt"

In [210]:
pdGroupes=pd.read_csv(nomGroupes,sep="\t",names=["Nom","Prénom","Groupe"],encoding="utf8")
pdGroupes["Name"]=pdGroupes["Prénom"]+" "+pdGroupes["Nom"]
pdGroupes=pdGroupes.groupby(by="Groupe")["Name"].apply(list).reset_index()
groupeEtudiants=pdGroupes["Name"].to_dict()
pdGroupes[["Name"]]

,Name
0,"[RACHEL BALABUCH, ALEXIA CLEMENT, ANAIS HUGUET..."
1,"[ANGELIQUE BECKER, LUCAS PRUNIER, PATRICIA RIC..."
2,"[MAUD BONNEFON, ANNE-SOPHIE DACKO, CARLA DELUR..."
3,"[LILI AGUER, ESTELLE BAUDIS, AXELLE DE OLIVEIR..."
4,"[CLARA BOLZEC, LEONIE CHANOIT, CAMILLE LANDAIS]"
5,"[MARINE BETHOULE, CHLOE DONATIEN, TULLIA PASSA..."
6,"[HOUYAM LAHSIBI, CHLOE LALANNE, JULIE PAULY]"
7,"[MAEVA BAYBAUD, CHLOE LE BACCON, NOLWENN MAZIE..."
8,"[CLARA CASSUS, MATTHIEU DELMOTTE, CORENTIN VENDE]"
9,"[JEAN-ELIE DURAND, MORGANE VASSAL, DIANE VIEIRA]"


In [96]:
htmlTable=etree.parse(nom,parser)

In [97]:
notes={}
nbLigne=0
for numLigne,ligne in enumerate(htmlTable.findall("tbody/tr")):
#    print numLigne,etree.tostring(ligne, pretty_print=True)
    if nbLigne==0:
        # sélection de td avec participant dans class
        selParticipant=cssselect.GenericTranslator().css_to_xpath('td.participant')
        participant= ligne.xpath(selParticipant)[0]
        nbLigne=int(participant.get("rowspan"))-1
        nomEtudiant=participant.find("span").text
        notes[nomEtudiant]=[]
        print
        print "Etudiant",nomEtudiant
    else:
        nbLigne-=1
#    print "il reste %d lignes"%nbLigne
    evaluateur=ligne.find('td[@class="receivedgrade notnull cell c2"]')
    selEvaluateur=cssselect.GenericTranslator().css_to_xpath('td.receivedgrade.notnull')
    evaluateurListe= ligne.xpath(selEvaluateur)
    if evaluateurListe:
        evaluateur=evaluateurListe[0]
        nomEvaluateur= evaluateur.find("div/span/span[@class='fullname']").text
        noteEvaluateur=evaluateur.find("div/a/span[@class='grade']").text
        notes[nomEtudiant].append((nomEvaluateur,noteEvaluateur))
        print "\tEvaluateur",nomEvaluateur, noteEvaluateur


Etudiant SONIA AANAT
	Evaluateur ANGELIQUE BOKOLOMBE 15
	Evaluateur CHLOE DONATIEN 16
	Evaluateur SONIA AANAT 18
	Evaluateur CHLOE ANDRE 20

Etudiant LILI AGUER
	Evaluateur MARINE BETHOULE 16
	Evaluateur ELISA ROBERT ELDUAYEN 18
	Evaluateur CYNTHIA TAJAN 18
	Evaluateur LILI AGUER 20

Etudiant CHLOE ALARCON
	Evaluateur CLARA CASSUS 13
	Evaluateur LILI AGUER 16
	Evaluateur CHLOE LE BACCON 19

Etudiant CHLOE ANDRE
	Evaluateur MARIE GUYOT 12
	Evaluateur MAUD BONNEFON 19
	Evaluateur GRAZIELLA GAGLIANO 18
	Evaluateur CHLOE ANDRE 20

Etudiant MARION ARAMENDI
	Evaluateur MARION ARAMENDI 18
	Evaluateur LAURA DESPORT 16
	Evaluateur HOUYAM LAHSIBI 17
	Evaluateur HIPPOLYTE MOREAU 20

Etudiant RACHEL BALABUCH
	Evaluateur MAJDA JOUHADI 15
	Evaluateur ANETA KIRCHNEROVA 20
	Evaluateur RACHEL BALABUCH 20
	Evaluateur QUENTIN BROUSSE 20

Etudiant SRILAKSHMI BALARD
	Evaluateur ESTELLE ROCHE 15
	Evaluateur ORIANE TOINARD 13
	Evaluateur SRILAKSHMI BALARD 16
	Evaluateur CAMILLE LANDAIS 19

Etudiant ESTELLE 

In [98]:
tabEvaluation=pd.DataFrame(columns=["Etudiant","Evaluateur","Note"])
i=0
for etudiant in notes:
    for evaluation in notes[etudiant]:
        tabEvaluation.loc[i]=[etudiant,evaluation[0],int(evaluation[1])]
        i+=1

In [99]:
tabEvaluateur=tabEvaluation.groupby(by=["Evaluateur"],as_index=False).agg({"Note":["mean","std"]})
tabEvaluateur.columns=["Evaluateur","Moyenne","Ecart"]
tabEvaluateur[tabEvaluateur["Evaluateur"]=="ANAIS HUGUET"]["Ecart"].values[0]

0.95742710775633821

In [174]:
moyenne=12.5
ecartType=2.5
individuNote={}
etuAbsents=["SIWEI LIU","CLEMENCE CALDERON"]
def moyEvaluateur(evaluateur):
    result=tabEvaluateur[tabEvaluateur["Evaluateur"]==evaluateur]["Moyenne"]
    if result.values:
        return result.values[0]
    else:
        return None

def coefEvaluateur(evaluateur):
    result=tabEvaluateur[tabEvaluateur["Evaluateur"]==evaluateur]["Ecart"]
    if result.values:
        coef=result.values[0]
        if coef<=0.5:
            return 0.5
        else:
            return coef
    else:
        return 0
    
    
def pondererNotes(x):
    x["NoteCorr"]=x["Note"]
    etudiant=x["Etudiant"].values[0]
#    print etudiant,"\t"
    coefs=0
    for i,element in x["Evaluateur"].iteritems():
        coefEval=coefEvaluateur(element)
        moyEval=moyEvaluateur(element)
#        print "\t",element, coefEval, moyEval, x["Note"][i]
        coefs+=coefEval
        x["NoteCorr"][i]=(x["Note"][i]-moyEval)*ecartType/coefEval
#        print "\t",x["NoteCorr"][i]
#    print round(x["NoteCorr"].mean()+moyenne,1)
#    print 
    individuNote[etudiant]=round(x["NoteCorr"].mean()+moyenne,1)
    return round(x["NoteCorr"].mean()+moyenne,1)
    
tabEvaluation.groupby(by=["Etudiant"],as_index=False).apply(pondererNotes)
groupesNote={}
for nGroupe in groupeEtudiants:
    mGroupe=0
    mIndividus=0
    for etu in groupeEtudiants[nGroupe]:
        if etu in etuAbsents:
            print etu           
        else:
#            print etu,individuNote[etu]
            mGroupe+=individuNote[etu]
            mIndividus+=1
#    print nGroupe+1,round(float(mGroupe)/mIndividus,1)
#    print 
    groupesNote[nGroupe+1]=round(float(mGroupe)/mIndividus,1)

SIWEI LIU
CLEMENCE CALDERON


In [189]:
pdGroupes=pd.DataFrame.from_dict(groupesNote,orient="index")
pdGroupes.columns=["Brut"]
pdGroupes.mean().values[0]

12.458333333333334

In [206]:
pdGroupes["Note"]=((pdGroupes["Brut"]-pdGroupes.mean().values[0])/pdGroupes.std().values[0]*2.45+12)
pdGroupes["Note"]=pdGroupes["Note"].round()
pdGroupes["Note"]

1     14.0
2     13.0
3     12.0
4     15.0
5     11.0
6     12.0
7     13.0
8     17.0
9     14.0
10    12.0
11    10.0
12    15.0
13    13.0
14     9.0
15    12.0
16     9.0
17     7.0
18    12.0
19    15.0
20    10.0
21    11.0
22    14.0
23    12.0
24     7.0
Name: Note, dtype: float64